In [ ]:
import os
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# HYPERPARAMETERS
LEARNING_RATE = 0.001
NUM_EPOCHS = 100

Part 1: Create Grumpy Cat Dataset

In [9]:
class CustomDataset(Dataset):
    def __init__(self, dir, transform = None):
        self.total_imgs: list[Image.Image] = []
        for file in os.listdir(dir):
            self.total_imgs.append( Image.open(file) )
        self.transform = transform
    
    def __len__(self) -> int:
        return len(self.total_imgs)
    
    def __getitem__(self, idx) -> Image.Image:
        return self.total_imgs[idx]

Part 2: Data Augmentation

In [10]:
def get_transform(mode: str) -> nn.Module:
    if mode == "simple":
        return transforms.Compose([
            transforms.Resize((64, 64), transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    elif mode == "deluxe":
        return transforms.Compose([
            transforms.Resize((64, 64), transforms.InterpolationMode.BICUBIC),
            transforms.RandomGrayscale(0.2),
            transforms.RandomRotation(180),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    raise NotImplementedError

Part 3: Implement the Discriminator of the DCGAN

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, noise_size=100):
        super(Discriminator, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(  3,  32, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d( 32,  64, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d( 64, 128, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d(256, noise_size, kernel_size=4, stride=2, padding=1),
        )
    
    def forward(self, z) -> torch.Tensor:
        return self.layers(z).squeeze()

Part 4: Implement the Generator of the DCGAN

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_size=100):
        super(Generator, self).__init__()
        self.layers = nn.Sequential(
            nn.ConvTranspose2d(noise_size, 256, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d( 64, 32, kernel_size=4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d( 32,  3, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )
    
    def forward(self, z) -> torch.Tensor:
        return self.layers(z)

Part 5: Training Loop

In [ ]:
def training_loop(dataloader: DataLoader):
    generator = Generator()
    discriminator = Discriminator()

    g_optimizer = optim.Adam(generator.parameters(), lr=LEARNING_RATE)
    d_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        for data in dataloader:

            d_optimizer.zero_grad()
            